# Carolyn's scratch notebook to work through toilet data


In [3]:
#load libraries

import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt

### Import Data from LA Parks and LA CAN

In [4]:
# upload park bathroom data
parks = pd.read_csv('MiscData/parkslatlon.csv')
parks.dropna(inplace = True) #drop the few NaNs

#upload LACan bathroom, handwashing stations, etc data
lacan = pd.read_csv('LACAN Data/allstations_LACAN.csv')

In [5]:
# convert to geodataframes and project
parksGdf = gpd.GeoDataFrame(parks, geometry=gpd.points_from_xy(parks.lon, parks.lat, 
                                                               crs='EPSG:4326'))

lacanGdf = gpd.GeoDataFrame(lacan, geometry=gpd.points_from_xy(lacan.X, lacan.Y, 
                                                               crs='EPSG:4326'))

In [20]:
geotoiletsGdf = gpd.read_file('MyGeodataCloud_Public_Toilets_or_Bathrooms/toilets_point.geojson')

DriverError: MyGeodataCloud_Public_Toilets_or_Bathrooms/toilets_point.geojson: No such file or directory

In [19]:
#parksGdf.head(2)

### Prepare data to concatonate
First, begin with parks data

In [ ]:
#drop unnecessary columns
parksGdf = parksGdf.drop(columns=['Unnamed: 0', 'address', 'lat', 'lon'])


In [ ]:
#add a column to specify type
parksGdf['type'] = 'LA Parks'

#rearrange columns if necessary - in this case, not necessary
#parksGdf = parksGdf[['name', 'geometry', 'type']]

In [10]:
parks.head()

,name,geometry,type
0,Augustus F. Hawkins Natural Park,POINT (-118.24786 34.01386),LA Parks
1,Bird Sanctuary,POINT (-118.29180 34.10518),LA Parks
2,Bishop Canyon,POINT (-118.23659 34.07990),LA Parks
4,Branford Recreation Center,POINT (-118.42319 34.23351),LA Parks
6,Chavez Ravine Arboretum,POINT (-118.24115 34.06665),LA Parks


In [12]:
#drop unnecessary columns from LACAN
lacanGdf = lacanGdf.drop(columns=['X', 'Y', 'description', 'gx_media_links'])

In [13]:
#rearrange columns
lacanGdf = lacanGdf[['name', 'geometry', 'type']]

### Concatenate

In [14]:
#concatenate the two gdfs
lacanparks = pd.concat([parksGdf, lacanGdf], axis=0)

In [15]:
lacanparks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 283 entries, 0 to 255
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   name      283 non-null    object  
 1   geometry  283 non-null    geometry
 2   type      283 non-null    object  
dtypes: geometry(1), object(2)
memory usage: 8.8+ KB


In [ ]:
fig, ax = plt.subplots(figsize=(20,20))

# add the layer with ax=ax in the argument 
parksGdf.plot(ax=ax,
               color='r'
              )

# add a basemap
ctx.add_basemap (ax,source=ctx.providers.CartoDB.Positron)

## Section 1. Attempts to scrape restroom data from LA Parks

I'm mostly trying to follow the lecture notebook for '7 Scraping Craigslist'

In [ ]:
# using the laparks.org page for Gladys Park which I confirmed has restrooms
import requests
from bs4 import BeautifulSoup

url = 'https://www.laparks.org/park/6th-gladys-street'
r = requests.get(url)

soup = BeautifulSoup(r.content)
print(soup.prettify())


In [ ]:
restrooms = soup.find_all(class_ = 'field field-name-field-public-info field-type-text-long field-label-above')
print(len(restrooms))

In [ ]:
restrooms[0]

In [ ]:
print(restrooms[0].find('div', class_= 'field-item even').text)

In [ ]:
hours = soup.find_all(class_ = 'field field-name-field-hours-of-operation field-type-text-long field-label-above')
print(len(hours))

In [ ]:
print(hours[0].find('div', class_= 'field-item even').text)

In [ ]:
# skipped a step in the lecture notebook about getting the URL
#will have to come back to it

In [ ]:
import pandas as pd
restroomList = [] # empty list that we can append to

for restroom in restrooms:

## Section 1.2 More Park stuff

In [ ]:
park_url = 'https://www.laparks.org/park/encino'
r = requests.get(park_url)
soup = BeautifulSoup(r.content)
print(soup.prettify())

In [ ]:
restrooms = soup.find_all(class_= 'field-label')
print (len(restrooms))

In [ ]:
restrooms [0]

In [ ]:
restrooms [1]

In [ ]:
restrooms[2]

In [ ]:
restrooms [3]

In [ ]:
restrooms [4]

In [ ]:
restrooms[5]

In [ ]:
restrooms[6]

In [ ]:
restrooms [7]

In [ ]:
restrooms[8]

In [ ]:
print(restrooms[6].find('div', class_= 'field-label').text)

In [ ]:
libraries = 'https://lapl.org/branches'
rlibraries = requests.get(libraries)
soup = BeautifulSoup(rlibraries.content)
print(soup.prettify())

## Section 3. County Library Data API

In [ ]:
#First I will create a pandas dataframe (countylib) of all LA County Library locations, including their name, hours, and the other information that is returned from the branch information API.

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.lacity.org", None)

results = client.get("a4nt-4gca", limit=2000)

countylib = pd.DataFrame.from_records(results)

In [ ]:
countylib.info()

In [ ]:
#geojson
import json
import requests
import pandas as pd

r = requests.get('https://data.lacounty.gov/resource/nrtn-ga3s.geojson')
library = r.json()
df = pd.DataFrame(censusdata[1:], columns=censusdata[0])
df.head()
https://data.lacounty.gov/resource/nrtn-ga3s.geojson

In [ ]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.lacounty.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.lacounty.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("nrtn-ga3s", limit=2000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [ ]:
results_df.info()

In [ ]:
results_df.head()